In [20]:
!pip install gradio librosa gTTS transformers sentencepiece

   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ------------------------------- -------- 786.4/991.5 kB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 991.5/991.5 kB 7.8 MB/s eta 0:00:00


In [1]:
import gradio as gr
import numpy as np
import librosa
import os
import uuid
from gtts import gTTS
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

c:\Users\erand\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\erand\AppData\Local\Programs\Python\Python310\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


c:\Users\erand\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\erand\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
asr_pipeline = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Option 1: English to Sinhala Translation
print("Loading English-Sinhala model...")
sinhala_tokenizer = AutoTokenizer.from_pretrained("thilina/mt5-sinhalese-english")
sinhala_model = AutoModelForSeq2SeqLM.from_pretrained("thilina/mt5-sinhalese-english")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading English-Sinhala model...


pytorch_model.bin:   3%|2         | 62.9M/2.39G [00:00<?, ?B/s]

c:\Users\erand\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(

In [4]:
def process_uploaded_audio(file_path):
    try:
        audio_data, sr = librosa.load(file_path, sr=16000)
        result = asr_pipeline(audio_data.astype(np.float32))
        english_text = result["text"]
        tokens = tokenizer(english_text, return_tensors="pt", padding=True, truncation=True)
        translated_tokens = model.generate(**tokens)
        spanish_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
        output_filename = f"translated_{uuid.uuid4().hex}.mp3"
        tts = gTTS(text=spanish_text, lang='es')
        tts.save(output_filename)
        return english_text, spanish_text, output_filename, "Translation complete"
    except Exception as e:
          return None, None, None, f"Error: {str(e)}"

In [5]:
with gr.Blocks(title="Speech to Speech Translator") as app:
    gr.Markdown("Speech to Speech Translator")
    gr.Markdown("Upload MP3 File or Record Your Own Voice")
    file_input = gr.Audio(type="filepath", label="Upload English MP3 File")
    translate_button = gr.Button("Translate to Spanish")
    english_textbox = gr.Textbox(label="English Transcription")
    spanish_textbox = gr.Textbox(label="Spanish Translation")
    audio_output = gr.Audio(label="Spanish Audio", type="filepath")
    status_output = gr.Textbox(label="Status")
    translate_button.click(
        fn=process_uploaded_audio,
        inputs = file_input,
        outputs = [english_textbox, spanish_textbox, audio_output, status_output]
    )
if __name__ == "__main__":
    app.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
